# Ajustement de relations d'échelle en Python avec LIRA

Les codes dans ce `git` offrent une interface `python` à la librairie `LIRA`, originalement écrite en `R`.
Ce notebook présente un guide d'utilisateur pour les modules principaux de la librairie.
Pour plus de détails scientifiques sur la modélisation probabiliste des relations masse-observable, voir [1, 2, 3].

# Installation des dépendances

## Sur MacOS

```sh
brew install R
brew install jags
pip3 install rpy2
```
Puis ouvrir une session `R` (en tapant `R` dans un terminal) et lancer:
```R
install.packages('lira')
```

## Sur les serveurs du LPSC

Les dépendances devraient déjà être installées, au moins sur `lpsc-nika2e`. Sinon, 

```sh
pip3 install --user rpy2
```
et (demander à Juan) :
```sh
sudo apt install R
sudo apt install rjags
```
Puis ouvrir une session `R` (en tapant `R` dans un terminal) et lancer:
```R
install.packages('lira')
```

# Notations

Ces codes utilisent au mieux les mêmes notations que `LIRA` pour les noms de variables. Lorsque ce n'est pas possible (`R` permettant d'utiliser `.` dans des noms de variables et `python` non), les `.` sont remplacés par des `_`.

La signification de chaque grandeur et son rôle dans la relation d'échelle est expliquée avec un grand niveau de détail dans les sections 2&3 de [2].
La correspondance entre les grandeurs physiques et les noms de variable est expliquée dans la Table 1 de [2], copiée ci-dessous.

![LIRA_table1](./LIRA_table1.png)

# Génération d'un échantillon et ajustement avec LIRA

Voici une illustration d'utilisation de l'interface `Python`, en utilisant un échantillon arbitraire avec les propriétés suivantes :

* $ Z \sim \mathcal{N}(0, 1) $ : pour des amas, correspond à un échantillon dont les masses suivent une distribution log-normale.
* $ X = Z $                    : le proxy de masse utilisé - e.g. masse hydrostatique ou masse lensing - est non-biaisé et non-dispersé.
* $ Y \sim \mathcal{N}(1 + X, 0.1^2) $                              : relation d'échelle avec $\alpha_{Y|Z}=1$, $\beta_{Y|Z}=1$, $\sigma_{Y|Z}=0.1$
* $ x \sim \mathcal{N}(X, 0.1^2), \; y \sim \mathcal{N}(Y, 0.1^2) $ : $x$ et $y$ sont des mesures de $X$ et $Y$ avec des incertitudes gaussiennes non-corrélées.


In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from fisc import Data

n = 100
truth = {
    "alpha.YIZ": 1.0, "beta.YIZ": 1.0, "sigma.YIZ.0": 0.1,
    "alpha.XIZ": 0.0, "beta.XIZ": 1.0, "sigma.XIZ.0": 0.0,
}

np.random.seed(123)
Z = np.random.normal(0.0, 1.0, n) # True latent variable, i.e. cluster (log) mass
X = np.random.normal( # True value of (log) mass estimator
    truth["alpha.XIZ"] + truth["beta.XIZ"] * Z, truth["sigma.XIZ.0"]
)
Y = np.random.normal( # True value of (log) observable, i.e. mass proxy
    truth["alpha.YIZ"] + truth["beta.YIZ"] * Z, truth["sigma.YIZ.0"]
)

err_x = np.ones(n) * 0.1 # uncertainty on x
err_y = np.ones(n) * 0.1 # uncertainty on y
x = np.random.normal(X, err_x) # x = noisy measurement of X
y = np.random.normal(Y, err_x) # y = noisy measurement of Y

d = Data(x, y, err_x, err_y) # The structure to call LIRA

La classe `Data` du module `fisc.py` contient des méthodes de classe permettant notamment de manipuler les données.
On peut notamment visualiser le jeu de données généré :

In [2]:
fig, ax = d.plot_data(style="errb")
d.plot_alphabeta(ax, truth["alpha.YIZ"], truth["beta.YIZ"], setlims=True, color="k", ls="--", label="Truth") # truth line
ax.legend(frameon=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

L'ajustement avec `LIRA` peut être réalisé avec la méthode `fit_lira`. La documentation de cette méthode explique la signification des arguments et offre des exemples :

```python
fit_lira(self, nmix, nsteps, lira_args={})

Use the LIRA R package to perform the fit.

Parameters
----------
nmix : int
    Number of gaussians in the mixture model;
nsteps : int
    Number of steps to perform in the MCMC;
lira_args : dict
    Arguments to pass to LIRA. The syntax is the same
    as for the R function.

Returns
-------
chains : DataFrame
    The MCMC chains in the parameter space

Examples
--------
d.fit_lira(3, 10_000, lira_args={"sigma.YIZ.0": "prec.dgamma"} )
d.fit_lira(3, 10_000, lira_args={
    "sigma.XIZ.0": 0.0, "sigma.YIZ.0": "dunif(0.0, 1.0)"
})
```

Pour l'exemple précédent, en considérant les priors par défaut de LIRA pour tous les paramètres :

In [3]:
chains = d.fit_lira(3, 25000)

R[write to console]: Loading required package: coda

R[write to console]: Loading required package: rjags

R[write to console]: Linked to JAGS 4.3.0

R[write to console]: Loaded modules: basemod,bugs



[1] Running: nsteps=25000, nmix=3
[1] "No threshold detected"


R[write to console]: module mix loaded



  |**************************************************| 100%

Iterations = 6252:31251
Thinning interval = 1 
Number of chains = 1 
Sample size per chain = 25000 

1. Empirical mean and standard deviation for each variable,
   plus standard error of the mean:

                          Mean        SD  Naive SE Time-series SE
alpha.YIZ            1.007e+00 1.568e-02 9.919e-05      5.304e-04
beta.YIZ             1.009e+00 1.368e-02 8.654e-05      4.428e-04
mu.Z.0               1.508e+03 5.172e+03 3.271e+01      1.869e+03
mu.Z.0.mixture[2]    1.444e-02 1.140e-01 7.207e-04      8.840e-04
mu.Z.0.mixture[3]    8.765e+00 5.766e+03 3.647e+01      1.194e+02
pi[1]                9.057e-03 9.097e-03 5.753e-05      2.515e-04
pi[2]                9.817e-01 1.301e-02 8.229e-05      3.411e-04
pi[3]                9.237e-03 9.418e-03 5.957e-05      2.429e-04
sigma.YIZ.0          5.018e-02 2.685e-02 1.698e-04      1.811e-03
sigma.Z.0            1.898e+44 4.656e+45 2.944e+43      1.767e+44
sigma.Z.0.mixtu

`LIRA` donne par défaut des informations intéressantes sur la distribution échantillonnée.
On peut l'examiner plus visuellement, par exemple à l'aide de `chainconsumer` [4] :

In [4]:
from chainconsumer import ChainConsumer

cc = ChainConsumer()
cc.add_chain(chains[["alpha.YIZ", "beta.YIZ", "sigma.YIZ.0"]]) # only the 3 parameters of interest
cc.configure(cmap="Spectral_r", shade_gradient=0.0, shade_alpha=0.3) # prettify!
cfig = cc.plotter.plot(figsize=(7, 7), truth=truth)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Et dans l'espace $(x, y)$ d'intérêt :

In [5]:
fig, ax = d.plot_data(style="errb")
d.plot_alphabeta(ax, truth["alpha.YIZ"], truth["beta.YIZ"], setlims=True, color="k", ls="--", label="Truth") # truth line
d.plot_alphabeta(ax, np.median(chains["alpha.YIZ"]), np.median(chains["beta.YIZ"]), color="tab:blue", label="LIRA") # relation at median of chains
d.plot_alphabeta(ax, chains["alpha.YIZ"], chains["beta.YIZ"], color="tab:blue") # chains confidence intervals
ax.legend(frameon=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Pour changer des options, comme le prior sur un paramètre, il suffit de l'injecter dans la fonction `fit_lira` sous forme du dictionnaire `lira_args`.
Ce dictionnaire doit être structuré comme
```json
{"nom de la variable dans LIRA": valeur de la variable dans LIRA}
```

Par exemple, pour définir :

* Le prior sur $\sigma_{Y|Z}$ comme une distribution uniforme entre 0 et 1 ;
* Le prior sur $\sigma_{X|Z}$ comme une distribution de Dirac en 0 (i.e. fixer $\sigma_{X|Z} = 0$),

l'appel à `LIRA` s'écrit (en `R`):
```R
lira(..., sigma.YIZ.0="dunif(0.0, 1.0)", sigma.XIZ.0=0.0)
```
En `python`:
```python
chains = d.fit_lira(1, 25000, lira_args={"sigma.YIZ.0": "dunif(0.0, 1.0)", "sigma.XIZ.0": 0.0})
```

# Échantillons d'amas

Le module `cluster_catalog.py` offre des outils pour générer aléatoirement des échantillons d'amas pour en étudier la relation d'échelle.
Ces échantillons peuvent ensuite être transférés au module `fisc.py` pour l'ajustement avec LIRA.

La classe `ClusterCatalog` permet de construire un échantillon d'amas. Elle peut être initialisée de plusieurs façons, selon les propriétés recherchées pour l'échantillon :

In [6]:
import cluster_catalog

# 1) À partir de valeurs de z et M_500
z = np.random.uniform(0.5, 0.9, 200)
M_500 = np.random.uniform(3e14, 11e14, 200)
catalog = cluster_catalog.ClusterCatalog(z, M_500)
fig, ax = catalog.plot_mz()
fig.suptitle("From $M$ and $z$ values")

# 2) À partir d'une table `astropy.table` contenant z et M_500
from astropy.table import Table
t = Table.read("./catalog_universe_1e5.fits")
# Ce fichier contient un échantillon de 10^5 amas suivant une fonction de masse Tinker+10,
# avec 2e14 < M500/Msun < 2e15, 0 < z < 1.2
catalog = cluster_catalog.ClusterCatalog.from_table(t)
fig, ax = catalog.plot_mz()
fig.suptitle("From table")

# 3) À partir d'une fonction de masse
# Cette méthode ne fonctionne plus avec la nouvelle version de la librairie hmf :(
# Elle n'était de toute façon pas optimale, il serait mieux de réécrire un générateur
# basé sur de l'inverse CDF sampling

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, 'From table')

Les valeurs d'observable associées peuvent être calculées, en supposant une relation d'échelle en loi de puissance dont les paramètres doivent être choisis par l'utilisateur.
Par exemple, pour calculer les valeurs de $Y_{500}$ à partir de la relation d'échelle *Planck* :

$$ E^{-2/3}(z) D_A^2 \frac{Y_{500}}{10^{-4} \, {\rm Mpc^2}} = 10^{-0.19} \times \left[\frac{M_{500}}{6 \times 10^{14} \, M_\odot}\right]^{1.79} \pm 0.075$$

In [7]:
catalog.to_observable(alpha=-0.19, beta=1.79, sigma=0.075)

On peut également ajouter des incertitudes de mesure aux valeurs d'observable et de masse. Par exemple, en considérant les grandeurs observées $(x, y)$ comme des estimateurs dispersés avec une incertitude de 10%, avec une corrélation entre 80% et 85% :

In [8]:
catalog.measurement_errors(err_frac_Y=(0.1, 0.0), err_frac_M=(0.1, 0.0), corr=(0.8, 0.85))

## Sélection de sous-échantillons

Il est aussi possible d'isoler seulement une partie d'un échantillon selon les valeurs d'observable et de redshift.
Pour cela, il faut utiliser la classe `Box` du module `cluster_catalog.py` et la méthode `select_from_boxes` de la classe `ClusterCatalog`.
Par exemple, pour selectionner à partir de l'échantillon précédent (contenu dans `./catalog_universe_1e5.fits`) un sous-échantillon :

* De 100 amas
* Dont les valeurs de redshift sont compris entre 0.5 et 0.9 (gamme couverte par le LPSZ de NIKA2)
* Dont les valeurs d'observable $\tilde{Y}$ sont comprises entre 0.21 et 2.2 (gamme couverte par le LPSZ de NIKA2)

Il faut appliquer l'algorithme suivant :

1. Lire le catalogue d'amas pour obtenir les valeurs de masse et redshift
2. Appliquer une relation d'échelle pour obtenir des valeurs d'observable associées à chaque amas
3. Créer un objet `Box` correspondant à l'intervalle en redshift er observable voulu
4. Sélectionner 100 amas remplissant les conditions parmi l'échantillon initial

In [9]:
import numpy as np
from astropy.table import Table
import cluster_catalog

np.random.seed(123)
truth = {"alpha.YIZ": -0.19, "beta.YIZ": 1.79, "sigma.YIZ.0": 0.075}  # Relation d'échelle Planck

# 1.
t = Table.read("./catalog_universe_1e5.fits")
catalog = cluster_catalog.ClusterCatalog.from_table(t)

# 2.
catalog.to_observable(alpha=truth["alpha.YIZ"], beta=truth["beta.YIZ"], sigma=truth["sigma.YIZ.0"])
catalog.measurement_errors(err_frac_Y=(0.1, 0.0), err_frac_M=(0.1, 0.0), corr=(0.8, 0.85))

# 3.
mybox = cluster_catalog.Box(0.5, 0.9, 0.21, 2.2)

# 4.
new_catalog = catalog.select_from_boxes([mybox], [50])

fig, ax = new_catalog.plot_mz()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Pour ajuster la relation d'échelle de cet échantillon, il suffit de créer une instance `fisc.Data` à partir du catalogue.

In [10]:
from chainconsumer import ChainConsumer
from fisc import Data

d = Data.from_table(
    new_catalog(),
    x_obs="log_M_tilde",
    y_obs="log_Y_tilde",
    x_err="err_log_M_tilde",
    y_err="err_log_Y_tilde",
    y_threshold="log_Y_tilde_threshold", # voir Notes
    corr="corr",
)

chains = d.fit_lira(3, 25000)

fig, ax = d.plot_data(style="ellipse")
d.plot_alphabeta(ax, truth["alpha.YIZ"], truth["beta.YIZ"], setlims=True, color="k", ls="--", label="Truth") # truth line
d.plot_alphabeta(ax, np.median(chains["alpha.YIZ"]), np.median(chains["beta.YIZ"]), color="tab:blue", label="LIRA") # relation at median of chains
d.plot_alphabeta(ax, chains["alpha.YIZ"], chains["beta.YIZ"], color="tab:blue") # chains confidence intervals
ax.legend(frameon=False)

cc = ChainConsumer()
cc.add_chain(chains[["alpha.YIZ", "beta.YIZ", "sigma.YIZ.0"]]) # only the 3 parameters of interest
cc.configure(cmap="Spectral_r", shade_gradient=0.0, shade_alpha=0.3) # prettify!
cfig = cc.plotter.plot(figsize=(7, 7), truth=truth)

[1] Running: nsteps=25000, nmix=3
[1] "Threshold detected"
  |**************************************************| 100%

Iterations = 6252:31251
Thinning interval = 1 
Number of chains = 1 
Sample size per chain = 25000 

1. Empirical mean and standard deviation for each variable,
   plus standard error of the mean:

                           Mean        SD  Naive SE Time-series SE
alpha.YIZ            -1.201e-01 9.210e-02 5.825e-04      6.982e-03
beta.YIZ              2.921e+00 6.421e-01 4.061e-03      3.996e-02
mu.Z.0               -1.666e-01 2.938e-02 1.858e-04      1.686e-03
mu.Z.0.mixture[2]     1.271e+01 5.758e+03 3.641e+01      4.350e+01
mu.Z.0.mixture[3]     1.689e+02 1.230e+02 7.780e-01      9.895e+01
pi[1]                 9.469e-01 9.304e-02 5.885e-04      1.454e-02
pi[2]                 1.905e-02 1.837e-02 1.162e-04      4.540e-04
pi[3]                 3.407e-02 9.162e-02 5.794e-04      1.560e-02
sigma.YIZ.0           8.022e-02 4.163e-02 2.633e-04      2.524e-03
sigma.Z.0   

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Sélection dans plusieurs boîtes : LPSZ

Pour sélectionner dans plusieurs boîtes, il suffit de créer plusieurs objets `Box`. Les 10 boîtes du LPSZ sont déjà créées dans `cluster_catalog.boxes_lpsz`.
La cellule de code suivante réalise une analyse complète telle que présentée en [3], section 7.5.1 (schématisée en figure 7.3) :

1. Sélection d'un échantillon suivant une fonction de masse (lue à partir du catalogue déjà existant)
2. Application d'une relation d'échelle fiducielle, correspondant à celle de *Planck*, + incertitudes de mesure
3. Sélection en boîtes du LPSZ
4. Ajustement de la relation d'échelle

In [11]:
import numpy as np
from astropy.table import Table
from chainconsumer import ChainConsumer
from fisc import Data
import cluster_catalog

truth = {"alpha.YIZ": -0.19, "beta.YIZ": 1.79, "sigma.YIZ.0": 0.075}  # Relation d'échelle Planck
np.random.seed(123)

# 1.
t = Table.read("./catalog_universe_1e5.fits")
catalog = cluster_catalog.ClusterCatalog.from_table(t)

# 2.
catalog.to_observable(alpha=truth["alpha.YIZ"], beta=truth["beta.YIZ"], sigma=truth["sigma.YIZ.0"])
catalog.measurement_errors(err_frac_Y=(0.1, 0.0), err_frac_M=(0.1, 0.0), corr=(0.8, 0.85))

# 3.
boxes = cluster_catalog.boxes_lpsz  # les boîtes du LPSZ sont déjà écrites
new_catalog = catalog.select_from_boxes(boxes, [5 for _ in range(len(boxes))])  # 5 amas par boîte

# 4.
d = Data.from_table(
    new_catalog(),
    x_obs="log_M_tilde",
    y_obs="log_Y_tilde",
    x_err="err_log_M_tilde",
    y_err="err_log_Y_tilde",
    y_threshold=None, # voir Notes
    corr="corr",
)

chains = d.fit_lira(3, 25000)

# Plot échantillon dans le plan (masse, redshift)
fig, ax = new_catalog.plot_mz()

# Plot données / relation d'échelle réelle / relation d'échelle reconstruite
fig, ax = d.plot_data(style="ellipse")
d.plot_alphabeta(ax, truth["alpha.YIZ"], truth["beta.YIZ"], setlims=True, color="k", ls="--", label="Truth") # truth line
d.plot_alphabeta(ax, np.median(chains["alpha.YIZ"]), np.median(chains["beta.YIZ"]), color="tab:blue", label="LIRA") # relation at median of chains
d.plot_alphabeta(ax, chains["alpha.YIZ"], chains["beta.YIZ"], color="tab:blue") # chains confidence intervals
ax.legend(frameon=False)

# Corner plot distribution postérieure
cc = ChainConsumer()
cc.add_chain(chains[["alpha.YIZ", "beta.YIZ", "sigma.YIZ.0"]]) # only the 3 parameters of interest
cc.configure(cmap="Spectral_r", shade_gradient=0.0, shade_alpha=0.3) # prettify!
cfig = cc.plotter.plot(figsize=(7, 7), truth=truth)

Selection: Could only take 4 clusters (instead of 5) with 1.4 < y < 2.2 and 0.5 < z < 0.7.
Selection: Could only take 0 clusters (instead of 5) with 1.4 < y < 2.2 and 0.7 < z < 0.9.
[1] Running: nsteps=25000, nmix=3
[1] "No threshold detected"
  |**************************************************| 100%

Iterations = 6252:31251
Thinning interval = 1 
Number of chains = 1 
Sample size per chain = 25000 

1. Empirical mean and standard deviation for each variable,
   plus standard error of the mean:

                           Mean        SD  Naive SE Time-series SE
alpha.YIZ            -2.174e-01 1.774e-02 1.122e-04      5.576e-04
beta.YIZ              1.630e+00 1.507e-01 9.533e-04      6.676e-03
mu.Z.0               -4.819e+01 5.772e+03 3.651e+01      4.285e+01
mu.Z.0.mixture[2]    -1.009e-02 2.791e-02 1.765e-04      6.745e-04
mu.Z.0.mixture[3]    -2.970e+01 5.762e+03 3.644e+01      4.238e+01
pi[1]                 2.226e-02 2.046e-02 1.294e-04      5.355e-04
pi[2]                 9.563e

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Notes

Une sélection basée sur la valeur d'observable a pour conséquence un biais dans la relation d'échelle, connu sous le nom de biais de Malmquist.
Si la sélection est faite "dans une seule boite" (i.e. la sélection est seulement une coupure en observable), la valeur du seuil peut être utilisée pour corriger de ce biais dans la modélisation probabiliste de la relation d'échelle (voir explication en section 7.2.2 de [3]).
Pour cette raison, lors de l'appel de la méthode `select_from_boxes`, la limite inférieure de la (des) boîte(s) est stockée dans la variable `log_Y_tilde_threshold`.

Comme montré dans [3] (section 7.5.2), cette modélisation est inadaptée pour une sélection dans plusieurs boîtes.
Par conséquent, dans le cas d'une telle sélection, il est utile de réécrire la variable `log_Y_tilde_threshold` à `None`.

# Références

[1] M. Sereno, Manuel utilisateur de `LIRA` en `R` : https://cran.r-project.org/web/packages/lira/lira.pdf

[2] M. Sereno, Description scientifique du fonctionnement de `LIRA` : https://ui.adsabs.harvard.edu/abs/2016MNRAS.455.2149S/abstract

[3] F. Kéruzoré, Manuscrit de thèse, https://tel.archives-ouvertes.fr/tel-03555821

[4] Documentation ChainConsumer, https://samreay.github.io/ChainConsumer/